<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Flux-observability" data-toc-modified-id="Flux-observability-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Flux observability</a></span><ul class="toc-item"><li><span><a href="#Definitions" data-toc-modified-id="Definitions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Definitions</a></span></li></ul></li><li><span><a href="#Read-in-Neurospora--EMU-Network" data-toc-modified-id="Read-in-Neurospora--EMU-Network-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read in Neurospora  EMU Network</a></span></li><li><span><a href="#Calculate-active-fluxes-of-EMU-network-from-glucose-to-carbon-dioxide" data-toc-modified-id="Calculate-active-fluxes-of-EMU-network-from-glucose-to-carbon-dioxide-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Calculate active fluxes of EMU network from glucose to carbon dioxide</a></span></li><li><span><a href="#Calculate-EMU-pathways-of-active-fluxes" data-toc-modified-id="Calculate-EMU-pathways-of-active-fluxes-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Calculate EMU pathways of active fluxes</a></span><ul class="toc-item"><li><span><a href="#Visualize-EMU-pathways-for-glucose-to-carbon-dioxide" data-toc-modified-id="Visualize-EMU-pathways-for-glucose-to-carbon-dioxide-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Visualize EMU pathways for glucose to carbon dioxide</a></span></li></ul></li></ul></div>

# Flux observability

 [Crown and Antoniewicz, 2012](http://linkinghub.elsevier.com/retrieve/pii/S1096717611001273)  provide an algorithm for determining which fluxes are observable given a selection of tracers for 13C-metabolic flux analysis. 
## Definitions

* **Isotopic Labeling State** represents  a substrate with a mixture of labels where the location of each labeled atom is known.   To fully specify the isotopic labeling state of the  mixture, both percentage of each component of the mixture and which atoms are labeled in each component must be described.
* **Mass Isotopomer Distribution**  represents a substrate witha mixture of labels where the location of each labeled atom is not known. For a molecule with $n$ atoms, $M_0,M_1,\ldots,M_n$ specifies the percentage of atoms that are unlabeled, 1-labeled, up to fully labeled.
* **Elementary Metabolite Units (EMU)** are distinct subsets of the compound's atoms.  There are 7 EMU's for a 3-atom metabolite $A$ shown in the figure from [Antoniewicz 2007](http://linkinghub.elsevier.com/retrieve/pii/S109671760600084X) below. The subscript in the first column and the shaded areas in the second column denote the atoms that are included in the EMU. The EMU size is the number of atoms included in the EMU. ![EMU_definitions](EMU_definition.png)
* An **EMU network** is a set of EMU's and the transitions between them.  The network can be restricted to EMUs of a particular size (Figure 3A), or it can be combined into a larger network composed of EMU's of all sizes (Figure 3B).
*  **EMU basis vector** can be viewed as a unique way for assembling substrate EMUs to form the product. By definition, EMU basis vectors have the same EMU size as the product. Ultimately, one can interpret the Mass Isotopic Distribution (MID) of the product as a linear combination of EMU basis vector MIDs, where the coefficients are solely a function of free fluxes. The coefficients quantify the fractional contribution of each EMU basis vector to the product. Thus, by definition, the sum of the coefficients must equal one. 
* In addition to the EMU basis vector, Crown et al also define an **EMU pathway**. Each EMU basis vector specifies a unique EMU pathway from substrate(s) to product. 
* An **Elementary mode (ELMO)** of a network is a minimal cardinality set of reactions that form a steady-state. Minimal cardinality means that no steady-state flux distribution can be a subset of an elementary mode.  The set of elementary modes forms a basis for steady-state flux space.
* Assertion: The elementary modes of the EMU network are the EMU pathways. Evidence for this assertion is provided below.

In [1]:
import gurobipy
from cobra.util.solver import linear_reaction_coefficients
from IPython.display import display, Image, Markdown, Latex, HTML
import pandas as pd
import numpy as np
import scipy, sympy
from cobra import Metabolite, Reaction, Model
import cobra
import sys, re,os
sys.path.append(os.path.join(os.environ['HOME'],'Projects/src'))
from pyefm import calculate_elementary_modes, calculate_minimum_cut_sets, utils
from d3flux import flux_map
from IPython.display import display, Image, SVG, Markdown,Latex, HTML
import sympy 
from itertools import chain
from cobra.flux_analysis.geometric import geometric_fba
from pyemu import extract_emu_network, get_em_description, display_rxns



# Read in Neurospora  EMU Network

In [15]:
emu_network = {}
emu_network['full'] = cobra.io.load_json_model(filename='Neurospora_EMU_network_size_{}.json'.format('full'))
emu_pwys = {}
for size in range(1,8):
    emu_network[size] = extract_emu_network( emu_network['full'], size )
    print("Number of EMU reactions of size {}: {}".format(size, len(emu_network[size].reactions)))
    cobra.io.save_json_model(emu_network[size],filename='Neurospora_EMU_network_size_{}.json'.format(size),
                             sort=True, pretty=True)
emu_network['full'].solver

Number of EMU reactions of size 1: 797
Number of EMU reactions of size 2: 541
Number of EMU reactions of size 3: 301
Number of EMU reactions of size 4: 158
Number of EMU reactions of size 5: 69
Number of EMU reactions of size 6: 27
Number of EMU reactions of size 7: 5


# Calculate active fluxes of EMU network from glucose to carbon dioxide

In [12]:
emu_network['full'].solver = 'gurobi'
#emu_network['full'].objective='EX_co2_e_1'
soln = emu_network['full'].optimize().to_frame()
soln['equation'] = [emu_network['full'].reactions.get_by_id(rxn).reaction for rxn in soln.index]
active_fluxes = soln[soln['fluxes'] !=0]
active_fluxes

,fluxes,reduced_costs,equation
6PGLter_b_8,10.0,0.0,6pgl_r_1 --> 6pgl_1
ACL_11,10.0,0.0,cit_5 --> accoa_1
ACONT_f_19,20.0,0.0,cit_6 --> icit_6
CITtam_b_18,10.0,0.0,cit_m_5 --> cit_5
CITtam_b_21,20.0,0.0,cit_m_6 --> cit_6
CO2t_b_0,60.0,0.0,co2_1 --> co2_e_1
CO2tm_b_0,40.0,0.0,co2_m_1 --> co2_1
CSm_15,10.0,0.0,accoa_m_2 --> cit_m_5
CSm_2,20.0,0.0,accoa_m_1 --> cit_m_6
ENO_f_2,20.0,0.0,2pg_1 --> pep_1


# Calculate EMU pathways of active fluxes

In [14]:
glc_to_co2_model = cobra.Model('glc_to_co2')
glc_to_co2_model.add_reactions([emu_network['full'].reactions.get_by_id(rxn)
                            for rxn in active_fluxes.index])
glc_to_co2_pwys =  calculate_elementary_modes( glc_to_co2_model,verbose=False,java_args='-Xmx1g')
glc_to_co2_pwys.to_csv('yeast_EMU_pathways_size_{}.csv'.format('full'))
HTML(glc_to_co2_pwys.to_html())

,6PGLter_b_8,ACL_11,ACONT_f_19,CITtam_b_18,CITtam_b_21,CO2t_b_0,CO2tm_b_0,CSm_15,CSm_2,ENO_f_2,ENO_f_4,ENO_f_5,EX_co2_e_1,EX_glc-D_e_1,EX_glc-D_e_2,EX_glc-D_e_3,EX_glc-D_e_4,EX_glc-D_e_5,EX_glc-D_e_6,FBA_f_0,FBA_f_10,FBA_f_4,FUM_f_8,G6PDH2er_9,G6Pter_f_11,GAPD_f_0,GAPD_f_1,GAPD_f_5,GLCt1_1,GLCt1_10,GLCt1_4,GLCt1_6,GLCt1_7,GLCt1_8,GND_5,HEX1_11,HEX1_12,HEX1_2,HEX1_3,HEX1_6,HEX1_9,ICL_1,MALS_1,MDH_f_0,MDH_f_6,PDHm_0,PDHm_1,PDHm_2,PFK_10,PFK_4,PFK_9,PGI_f_0,PGI_f_11,PGI_f_3,PGI_f_6,PGI_f_8,PGK_b_1,PGK_b_3,PGK_b_5,PGL_10,PGM_b_1,PGM_b_3,PGM_b_5,PPCK_0,PPCK_4,PYK_0,PYK_1,PYK_4,PYRt2m_f_1,PYRt2m_f_2,PYRt2m_f_5,SUCD1m_f_7,SUCFUMtm_18,SUCFUMtm_9,TA1_f_6,TA1_f_8,TPI_f_4
EM1,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EM2,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
EM3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
EM4,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0
EM5,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0
EM6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Visualize EMU pathways for glucose to carbon dioxide

In [16]:
em_model = {}

for em in glc_to_co2_pwys.index:
    em_model[em] = cobra.Model(em)
    em_model[em].add_reactions([glc_to_co2_model.reactions.get_by_id(rxn)
                                for rxn in glc_to_co2_pwys.columns
                                   if glc_to_co2_pwys.loc[em,rxn] != 0])
    display(Markdown('### EMU Pathway {} ({})\n'.format(em, get_em_description(glc_to_co2_pwys.loc[em]))))
    display(flux_map(em_model[em],
                     display_name_format=lambda x: str(x.id),
                     flux_dict={rxn.id: glc_to_co2_pwys.loc[em,rxn.id] 
                            for rxn in em_model[em].reactions}))





![EM6](EX_glc-D_e_1.png)
![EM5](EX_glc-D_e_2.png)
![EM1](EX_glc-D_e_3.png)
![EM2](EX_glc-D_e_4.png)
![EM4](EX_glc-D_e_5.png)
![EM3](EX_glc-D_e_6.png)